In [ ]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns

# Đọc dữ liệu

In [ ]:
df_car = pd.read_csv("preprocessed_data.csv")

In [ ]:
df_car.head()

In [ ]:
#drop các trường ít tương quan

df_car.drop(
    [
        'url',
        'date',
        'out_color',
        'in_color',
        'num_door',
        'num_seat',
        'gear_type',
        'drive_type',
        'cylinder_capacity'
    ],
    axis=1,
    inplace=True
)

In [ ]:
# thêm trường tuổi dựa vào năm sx
df_car['age'] = datetime.datetime.now().year - df_car['model_year']

In [ ]:
df_car

# Phân tích used_distance

Vấn đề: 
- outliers
- xe mới (status = "Xe mới") có used_distance > 0
- xe cũ (status = "Xe đã dùng") có used_distance = 0

### outlier

thông thường ngoài thực tế số km đi đc trong 1 năm ko quá 15 vạn (100000 km/năm hoặc 275km/ngày). ta có thể tính số km trung bình/năm, nếu vượt quá 10 v có thể coi là outliers (điền nhầm số km, hoặc điền nhầm model_year) -> loại bỏ

In [ ]:
# sắp xếp những xe có km nhiều nhất
df_car.sort_values(by="used_distance", ascending=False).used_distance

In [ ]:
# thêm trường avg_mileage, là số km/năm
df_car['avg_mileage'] = df_car["used_distance"]/df_car['age']
df_car.head()

In [ ]:
# lọc những xe trên 10v
df1 = df_car[df_car.avg_mileage < 100000]

In [ ]:
len(df_car)-len(df1)

### Xe đã dùng mà km=0

In [ ]:
print("tổng số xe: ", len(df1))
print("số xe đã dùng mà km=0: ", len(df1[(df1.status == "Xe đã dùng") & (df1.used_distance == 0)]))

Do số lượng số xe đã dùng mà km=0 khá nhiều (>25%) do đó cần tìm cách sửa giá trị cho hợp lý. Có 1 vài khả năng làm cho có lỗi sai:
- set nhầm status thành xe đã sử dụng
- ko set số km

Một số cách tiếp cận:
- C1: sửa lại toàn bộ sử dụng avg_mileage
- C2: Ta có thể cân nhắc: với các xe age < x năm (x nhỏ khoảng 1, 2, 3 năm) tỉ lệ cao là set nhầm status -> để yên used_distance và chuyển status thành xe mới. Với các xe >= x năm, tỉ lệ cao là ko set số km -> sửa lại used_distance thành avg_mileage.
- C3: nghĩ thêm các hướng tiếp cận khác

In [ ]:
# C1: sửa tất cả thành avg_milaege

# lọc những xe hợp lệ: xe đã dùng với used_distance >0
df_valid_used_distance = df1[(df1.status == "Xe đã dùng") & (df1.used_distance > 0)]

# sử dụng aggregate để tính trung bình km theo tuổi

df_mean_distance_by_age = df_valid_used_distance.groupby(["age"]).mean()
df_mean_distance_by_age


In [ ]:
# hàm lambda xử lý logic thay giá trị
def lambda_func(x):
    if x.status == "Xe đã dùng" and x.used_distance == 0:
        if x["age"]>0: 
            return df_mean_distance_by_age.loc[x["age"]].used_distance
        else:
            return x.used_distance
    else:
        return x.used_distance

In [ ]:
# thay giá trị, thêm cột mới clean_used_distance
df1["clean_used_distance"] = df1.apply(lambda x: lambda_func(x), axis=1)
df1.head()

### xe mới có used_distance > 0

Đối với trường hợp này, có 2 khả năng có thể xảy ra:
- set nhầm status thành xe mới
- set nhầm số km

In [ ]:
print("Số xe mới có used_distance > 0 là: ", len(df1[(df1.status == "Xe mới") & (df1.used_distance > 0)]))

Vì có ít xe bị sai kiểu này, ta có thể bỏ hết các xe này.

In [ ]:
df1 = df1.drop(df1[(df1.status == "Xe mới") & (df1.used_distance > 0)].index)

### xe có used_distance nhỏ bất thường

có nhiều xe có số km nhỏ (<100km) có khả năng cao là do điền bừa

In [ ]:
print("số xe có km nhỏ hơn 100 là: ", len(df1[(df1.used_distance>0) & (df1.used_distance<100)]))

Có thể select dữ liệu và check tay xem có khả năng là điền bừa ko. nếu là điền bừa có thể xóa đi, hoặc là thay thế bằng giá trị km TB theo age.

## Vẽ biểu đồ

In [ ]:
df_used_car = df1[df1.status=="Xe đã dùng"]
df_used_car.head()

In [ ]:
sns.relplot(x="clean_used_distance", y="price_vnd", data = df_used_car)

In [ ]:
X = df_used_car[["clean_used_distance"]]
Y = df_used_car["price_vnd"]

In [ ]:
len(X), len(Y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)

In [ ]:
model.intercept_

In [ ]:
model.coef_

In [ ]:
model.predict(np.array([[10000]]))

In [ ]:
model.predict(np.array([[100000]]))

### One-hot encoding

In [ ]:
a = pd.DataFrame({"job": ["a", "b", "a", "c"]})
a

In [ ]:
pd.get_dummies(data=a)

In [ ]:
top_five_popular_brand = df_used_car["brand"].value_counts().index[:5]
df_used_popular_brand_car = df_used_car[df_used_car["brand"].isin(top_five_popular_brand)]

In [ ]:
len(df_used_popular_brand_car)

In [ ]:
X = df_used_popular_brand_car[["brand", "clean_used_distance", "age"]]
Y = df_used_popular_brand_car["price_vnd"]

In [ ]:
len(X), len(Y)

In [ ]:
X_onehot = pd.get_dummies(data=X)
X_onehot

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_onehot, Y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train,y_train)

In [ ]:
coeff_parameter = pd.DataFrame(model.coef_, X_onehot.columns, columns=['Coefficient'])
coeff_parameter